## 0. 실행 전 체크

### 필요 사항
- PyRosetta 설치/라이선스가 현재 커널에서 정상 동작해야 함
- 입력 파일이 노트북 작업 폴더에 존재해야 함  
  - 예: `fold_test1_model_0.pdb` (AF3 결과 PDB)

### 권장 실행 순서
위에서 아래로 순서대로 셀 실행

# 입력: AlphaFold3 결과 CIF만 업로드해서 시작

- 업로드 파일: `fold_test1_model_0.cif`
- 1단계: BioPython으로 CIF → PDB 변환
- 2단계: 변환된 PDB를 PyRosetta 파이프라인 입력으로 사용

In [ ]:
import os
from pathlib import Path

# 프로젝트 루트를 노트북 위치 기준으로 설정 (notebooks/ → 프로젝트 루트)
REPO_ROOT = Path(".").resolve().parent
DATA_DIR = REPO_ROOT / "data" / "fold_test1"

INPUT_CIF = str(DATA_DIR / "fold_test1_model_0.cif")
OUTPUT_PDB = "fold_test1_model_0_from_cif.pdb"

print("REPO_ROOT:", REPO_ROOT)
print("INPUT_CIF:", INPUT_CIF)
assert os.path.exists(INPUT_CIF), f"CIF 파일이 없습니다: {INPUT_CIF}"
print("[OK] CIF exists")

In [ ]:
# BioPython CIF -> PDB 변환
# 필요 패키지: biopython (conda: conda install -c conda-forge biopython)

from Bio.PDB import MMCIFParser, PDBIO

def cif_to_pdb(cif_path: str, pdb_path: str, structure_id="AF3_MODEL"):
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure(structure_id, cif_path)

    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_path)

    return pdb_path

out = cif_to_pdb(INPUT_CIF, OUTPUT_PDB)
print("[OK] Converted CIF -> PDB:", out)
print("File size (bytes):", os.path.getsize(out))

In [ ]:
INPUT_PDB = OUTPUT_PDB
print("[OK] Pipeline input PDB set to:", INPUT_PDB)

# SSTR2–SST14 설계 데모 (Notebook)

## 목표
이 노트북은 AlphaFold3 결과로 얻은 SSTR2–SST14 복합체 구조를 기반으로:

1) 입력 PDB 로드 및 체인/서열 요약을 텍스트로 확인  
2) **펩타이드(길이 14) 체인 자동 탐지**  
3) **체인 표준화: A=리셉터, B=펩타이드** (A/B 하드코딩 위험 제거)  
4) **펩타이드만 Relax** (리셉터는 고정)  
5) **FastDesign 후보 20개 생성 + 스코어링(dG/dSASA + 안정성 proxy)**  
6) **FlexPepDock으로 후보 refine + 재스코어링**

## 데모에서 강조할 아키텍처 포인트
- **체인 표준화(Chain standardization)** 를 통해 Interface 분석 지표가 흔들리지 않게 만듭니다.
- FastDesign은 “설계 후보 생성”, FlexPepDock은 “정밀 검증(refine)” 역할로 분리합니다.
- 혈중 안정성(6~10일) 자체를 정확히 예측하진 않더라도, **stability/PK proxy 레이어**를 넣어 멀티 오브젝티브 선별 구조를 데모로 보여줍니다.

In [ ]:
# 1 기본 라이브러리
import os
import csv
from collections import defaultdict, OrderedDict

# 2 표/정렬 출력용
import pandas as pd
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.width", 140)

print("Libraries imported")

In [ ]:
# PyRosetta 로드
# 주의: 커널에서 init은 보통 1회만 수행하는 것이 안전합니다.
import pyrosetta
from pyrosetta import rosetta

pyrosetta.init("-mute all -relax:default_repeats 3")
print("PyRosetta initialized")

# 1. 입력 파일 지정

여기서는 AF3 결과 PDB(`fold_test1_model_0.pdb`)를 입력으로 사용합니다.

- 입력 PDB에는 체인 A/B가 입력 순서에 따라 뒤바뀔 수 있으므로
- **표준화 단계에서 A=리셉터, B=펩타이드로 강제**합니다.

In [ ]:
# 입력 PDB 파일명 (Cell 4에서 CIF→PDB 변환 결과로 설정됨)
# INPUT_PDB 는 이미 Cell 4에서 정의되어 있음 — 재정의하지 않고 확인만 수행
print("INPUT_PDB:", INPUT_PDB)
assert os.path.exists(INPUT_PDB), f"파일이 없습니다: {INPUT_PDB}"

# 2. PDB 텍스트 기반 체인 요약(사전 점검)

PyRosetta로 들어가기 전에, PDB 파일 자체(ATOM 라인 기반)에서
- 체인 ID
- 잔기 수(길이)
- residue 번호 범위
- 서열 일부
를 빠르게 확인합니다.

In [ ]:
# 3-letter -> 1-letter 매핑 (표준 아미노산 중심)
AA3_TO_1 = {
    "ALA": "A", "ARG": "R", "ASN": "N", "ASP": "D", "CYS": "C",
    "GLN": "Q", "GLU": "E", "GLY": "G", "HIS": "H", "ILE": "I",
    "LEU": "L", "LYS": "K", "MET": "M", "PHE": "F", "PRO": "P",
    "SER": "S", "THR": "T", "TRP": "W", "TYR": "Y", "VAL": "V",
    "MSE": "M",
}

def parse_pdb_residues(pdb_path: str):
    """PDB를 ATOM/HETATM 라인 기준으로 파싱하여 chain->residue list를 만듦"""
    chains = defaultdict(OrderedDict)
    with open(pdb_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                continue
            if len(line) < 27:
                continue
            chain_id = line[21].strip() or "?"
            resname = line[17:20].strip().upper()
            resseq_raw = line[22:26].strip()
            icode = line[26].strip()
            try:
                resseq = int(resseq_raw)
            except ValueError:
                continue
            key = (resseq, icode)
            if key not in chains[chain_id]:
                chains[chain_id][key] = resname
    return chains

def residues_to_seq(res_dict: OrderedDict):
    return "".join(AA3_TO_1.get(res3, "X") for res3 in res_dict.values())

def contiguous_ranges(res_keys):
    nums = [k[0] for k in res_keys]
    if not nums:
        return []
    ranges = []
    start = prev = nums[0]
    for n in nums[1:]:
        if n == prev + 1:
            prev = n
        else:
            ranges.append((start, prev))
            start = prev = n
    ranges.append((start, prev))
    return [f"{a}-{b}" if a != b else f"{a}" for a, b in ranges]

def summarize_pdb(pdb_path, show_seq="head"):
    chains = parse_pdb_residues(pdb_path)
    rows = []
    for cid, res_dict in chains.items():
        keys = list(res_dict.keys())
        seq = residues_to_seq(res_dict)
        if show_seq == "head":
            seq_out = seq[:60] + ("..." if len(seq) > 60 else "")
        elif show_seq == "full":
            seq_out = seq
        else:
            seq_out = ""
        rows.append({
            "chain": cid,
            "length": len(keys),
            "pdb_res_min": keys[0][0] if keys else None,
            "pdb_res_max": keys[-1][0] if keys else None,
            "ranges": ", ".join(contiguous_ranges(keys)),
            "seq": seq_out,
        })
    return pd.DataFrame(rows).sort_values(["chain"]).reset_index(drop=True)

df_input_summary = summarize_pdb(INPUT_PDB, show_seq="head")
df_input_summary

# 3. PyRosetta Pose 로딩 + 펩타이드 체인 탐지(길이==14)

데모 재현성을 위해 펩타이드를 **길이==14**로 강하게 탐지합니다.

- 히트가 1개가 아니면 오류 처리(입력 구조가 예상과 다름)

In [ ]:
pose = pyrosetta.pose_from_pdb(INPUT_PDB)
print("num_chains:", pose.num_chains())

def find_peptide_chain_pose(pose, peptide_len=14):
    info = []
    for ch in range(1, pose.num_chains()+1):
        seq = pose.chain_sequence(ch)
        info.append((ch, len(seq), seq))
    df = pd.DataFrame(info, columns=["pose_chain_id", "length", "sequence"])
    display(df)
    hits = [ch for ch, ln, seq in info if ln == peptide_len]
    if len(hits) != 1:
        raise RuntimeError(f"길이=={peptide_len} 체인 탐지 결과가 1개가 아닙니다: {hits}")
    return hits[0]

peptide_chain_id = find_peptide_chain_pose(pose, peptide_len=14)
print("peptide_chain_id:", peptide_chain_id)

# 4. 체인 표준화: A=리셉터, B=펩타이드로 강제 저장

여기가 아키텍처 핵심 결함(1) 해결 포인트입니다.

입력 순서/변환 과정에서 체인 A/B가 바뀌어도, 이 단계에서:
- 리셉터(=펩타이드가 아닌 체인들)을 먼저 붙이고
- 펩타이드를 뒤에 붙여

`standardized_raw.pdb`를 항상 **A=리셉터, B=펩타이드**로 만듭니다.

In [ ]:
import tempfile

def extract_chain_pose_by_dump(original_pose, chain_id: int):
    """단순/설명 가능한 방식: 전체 dump 후 특정 체인만 필터링해서 다시 로드"""
    tmp_full = tempfile.NamedTemporaryFile(suffix=".pdb", delete=False).name
    tmp_chain = tempfile.NamedTemporaryFile(suffix=f"_chain{chain_id}.pdb", delete=False).name
    original_pose.dump_pdb(tmp_full)

    first_res = original_pose.chain_begin(chain_id)
    pdbinfo = original_pose.pdb_info()
    chain_letter = pdbinfo.chain(first_res) if pdbinfo is not None else ""
    if not chain_letter or chain_letter.strip() == "":
        chain_letter = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"[chain_id-1]

    with open(tmp_full, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    with open(tmp_chain, "w", encoding="utf-8") as out:
        for line in lines:
            if (line.startswith("ATOM") or line.startswith("HETATM")) and len(line) > 21:
                if line[21] == chain_letter:
                    out.write(line)
            if line.startswith("TER"):
                out.write(line)
        out.write("END\n")

    new_pose = pyrosetta.pose_from_pdb(tmp_chain)
    os.remove(tmp_full)
    os.remove(tmp_chain)
    return new_pose

def standardize_to_AB(pose, peptide_chain_id, out_pdb="standardized_raw.pdb"):
    receptor_chains = [ch for ch in range(1, pose.num_chains()+1) if ch != peptide_chain_id]
    if not receptor_chains:
        raise RuntimeError("receptor chain이 없습니다.")

    rec_pose = extract_chain_pose_by_dump(pose, receptor_chains[0])
    for ch in receptor_chains[1:]:
        rec_pose.append_pose_by_jump(extract_chain_pose_by_dump(pose, ch), rec_pose.total_residue())

    pep_pose = extract_chain_pose_by_dump(pose, peptide_chain_id)
    rec_pose.append_pose_by_jump(pep_pose, rec_pose.total_residue())

    rec_pose.dump_pdb(out_pdb)
    print(f"[OK] standardized saved -> {out_pdb} (A=receptor, B=peptide)")
    return rec_pose

standard_pose = standardize_to_AB(pose, peptide_chain_id, out_pdb="standardized_raw.pdb")

# 5. 표준화 자동 검증(PASS/FAIL) + 체인/잔기 범위 출력

- 체인 A 존재
- 체인 B 존재
- 체인 B 길이==14
- 체인 A 길이>14

이 조건을 만족하면 “표준화 성공”으로 판단합니다.

In [ ]:
df_std = summarize_pdb("standardized_raw.pdb", show_seq="head")
display(df_std)

rowA = df_std[df_std["chain"] == "A"]
rowB = df_std[df_std["chain"] == "B"]

assert len(rowA) == 1, "체인 A가 없습니다"
assert len(rowB) == 1, "체인 B가 없습니다"
assert int(rowB["length"].iloc[0]) == 14, f"체인 B 길이가 14가 아닙니다: {rowB['length'].iloc[0]}"
assert int(rowA["length"].iloc[0]) > 14, f"체인 A 길이가 너무 짧습니다: {rowA['length'].iloc[0]}"

print("[PASS] standardized_raw.pdb: A=receptor, B=peptide(len=14)")

# 6. peptide-only Relax (리셉터 고정)

표준화 후에는:
- Chain A(=pose chain 1) = 리셉터
- Chain B(=pose chain 2) = 펩타이드

따라서 **펩타이드 체인(2)만 MoveMap을 풀어서** FastRelax를 수행합니다.

In [ ]:
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.core.select.residue_selector import ChainSelector

def relax_peptide_only(in_pdb="standardized_raw.pdb", out_pdb="standardized_relaxed.pdb", peptide_chain_number=2):
    pose = pyrosetta.pose_from_pdb(in_pdb)

    mm = MoveMap()
    mm.set_bb(False); mm.set_chi(False); mm.set_jump(False)

    pep_selector = ChainSelector(peptide_chain_number)
    pep_res = pep_selector.apply(pose)
    for i in range(1, pose.total_residue()+1):
        if pep_res[i]:
            mm.set_bb(i, True)
            mm.set_chi(i, True)

    scorefxn = pyrosetta.get_score_function()
    relax = FastRelax()
    relax.set_scorefxn(scorefxn)
    relax.set_movemap(mm)

    pre = scorefxn(pose)
    relax.apply(pose)
    post = scorefxn(pose)

    pose.dump_pdb(out_pdb)
    print(f"[OK] Relax saved -> {out_pdb}")
    print(f"score pre: {pre:.2f}  post: {post:.2f}")
    return pre, post

pre_score, post_score = relax_peptide_only("standardized_raw.pdb", "standardized_relaxed.pdb", peptide_chain_number=2)

# 7. stability/PK proxy 점수(데모용)

혈중 안정성(6~10일)을 정확히 예측하는 것은 별도 모델/실험이 필요하지만,
데모 아키텍처 관점에서는 **서열 기반 proxy 점수 레이어가 존재**하는 것이 중요합니다.

- cleavage_risk: K/R, 방향족(F/Y/W) 기반 가중
- pk_penalty: 소수성 과다 + 전하 과다 페널티(간이)

※ 실제 연구 단계에서는 PeptideCutter 같은 도구/모델을 붙여 고도화 가능

In [ ]:
HYDROPHOBIC = set(list("AILMFWVY"))
BASIC = set(list("KRH"))
ACIDIC = set(list("DE"))

def stability_pk_proxy_scores(seq: str):
    seq = seq.strip().upper()
    kr = sum(1 for x in seq if x in "KR")
    arom = sum(1 for x in seq if x in "FYW")
    cleavage_risk = 2.0 * kr + 1.0 * arom

    hyd = sum(1 for x in seq if x in HYDROPHOBIC)
    hydrophobic_fraction = hyd / max(len(seq), 1)

    pos = sum(1 for x in seq if x in BASIC)
    neg = sum(1 for x in seq if x in ACIDIC)
    net_charge_proxy = pos - neg

    pk_penalty = 5.0 * max(0.0, hydrophobic_fraction - 0.50) + 0.5 * abs(net_charge_proxy)
    return {
        "cleavage_risk": cleavage_risk,
        "hydrophobic_fraction": hydrophobic_fraction,
        "net_charge_proxy": net_charge_proxy,
        "pk_penalty": pk_penalty,
    }

orig_pose = pyrosetta.pose_from_pdb("standardized_relaxed.pdb")
orig_seq = orig_pose.chain_sequence(2)
print("Original peptide seq:", orig_seq)
stability_pk_proxy_scores(orig_seq)

# 8. Interface 분석(dG/dSASA)

InterfaceAnalyzerMover는 인터페이스 정의 옵션을 갖고 있으며(서로 배타),
우리는 **체인 표준화(A/B)**를 통해 “무엇을 측정하는 dG인지”가 흔들리지 않게 합니다.
(참고 문서: InterfaceAnalyzerMover 파라미터/상호배타 옵션 설명)
- https://docs.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/analysis/InterfaceAnalyzerMover

In [ ]:
from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover

def analyze_interface(pose):
    """
    표준화(A=receptor, B=peptide) 후 인터페이스 분석.
    InterfaceAnalyzerMover에 "A_B" 인터페이스 문자열을 사용하여
    jump number 의존성을 제거하고 체인 기반으로 안정적으로 측정.
    """
    iam = InterfaceAnalyzerMover("A_B")
    iam.set_pack_separated(True)
    iam.set_compute_packstat(True)
    iam.apply(pose)
    return iam.get_interface_dG(), iam.get_interface_delta_sasa()

test_pose = pyrosetta.pose_from_pdb("standardized_relaxed.pdb")
dG0, dSASA0 = analyze_interface(test_pose)
print(f"baseline dG={dG0:.2f} REU, dSASA={dSASA0:.2f}")

# 9. FastDesign 후보 20개 생성 + DataFrame으로 즉시 확인

- 수용체(리셉터) 고정
- 펩타이드 Cys(고리 유지: C3, C14 → 이황화결합) 고정
- 후보 20개 생성
- 각 후보에 대해 서열 / dG / dSASA / stability proxy / 정책 위반 여부를 테이블로 출력

### Design Position 선정 근거 (Somatostatin-14: `AGCKNFFWKTFTSC`)

| 위치 | 잔기 | Design? | 근거 |
|------|------|---------|------|
| 1 (A) | Ala | No | 표면, 기여도 낮음 |
| 2 (G) | Gly | **Yes** | 유연성 조절 여지 |
| 3 (C) | Cys | **LOCK** | 이황화결합 (C3-C14) |
| 4 (K) | Lys | No | 전하 보존 필요 |
| 5 (N) | Asn | No | 구조 안정화 |
| 6 (F) | Phe | **Yes** | 약효단 인접, 방향족 최적화 가능 |
| 7 (F) | Phe | **Yes** | 핵심 약효단, 대안 방향족 탐색 |
| 8 (W) | Trp | **Yes** | 핵심 약효단(가장 깊은 삽입), 고위험/고보상 설계 |
| 9 (K) | Lys | No | 핵심 약효단(가장 가까운 접촉 2.58Å), 양전하 보존 필수 |
| 10 (T) | Thr | **Yes** | 약효단, 수소결합 대안 탐색 |
| 11 (F) | Phe | No | 보존적 위치 |
| 12 (T) | Thr | No | 수소결합 유지 |
| 13 (S) | Ser | No | 고리 근접 |
| 14 (C) | Cys | **LOCK** | 이황화결합 (C3-C14) |

> **K9는 설계에서 제외**: Lys9은 SSTR2 포켓 ASP122와 2.58Å 염다리를 형성하며, 이 양전하 상호작용은 결합의 핵심. 변이 시 결합력 급감 예상.

In [ ]:
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import OperateOnResidueSubset, PreventRepackingRLT
from pyrosetta.rosetta.core.select.residue_selector import (
    ChainSelector, NotResidueSelector, ResidueNameSelector, AndResidueSelector, OrResidueSelector
)
# FastDesign은 protocols.relax 모듈에 위치 (denovo_design.movers 아님)
from pyrosetta.rosetta.protocols.relax import FastDesign

def peptide_seq(pose, peptide_chain_id=2):
    return pose.chain_sequence(peptide_chain_id)

def diff_positions(original, new):
    return [i for i, (o, n) in enumerate(zip(original, new), start=1) if o != n]

def build_task_factory_lock_receptor_and_cys(pose, peptide_chain_id=2):
    pep_selector = ChainSelector(peptide_chain_id)
    rec_selector = NotResidueSelector(pep_selector)

    cys_selector = ResidueNameSelector("CYS")
    pep_cys_selector = AndResidueSelector(pep_selector, cys_selector)

    cant_touch = OrResidueSelector(rec_selector, pep_cys_selector)

    tf = TaskFactory()
    tf.push_back(OperateOnResidueSubset(PreventRepackingRLT(), cant_touch))
    return tf

def fastdesign_candidates(input_pdb="standardized_relaxed.pdb", n=20, design_pos="2,6,7,8,10"):
    # design 허용 포지션(펩타이드 내부 1-based)
    # 3,14=Cys(이황화결합 LOCK), 9=Lys(핵심 염다리 보존)
    # 2=Gly(유연성), 6,7=Phe(방향족), 8=Trp(삽입 깊이), 10=Thr(수소결합 대안)
    design_positions = [int(x.strip()) for x in design_pos.split(",") if x.strip()]
    allowed = set(design_positions)

    base_pose = pyrosetta.pose_from_pdb(input_pdb)
    orig_seq = peptide_seq(base_pose, 2)

    rows = []
    os.makedirs("candidates", exist_ok=True)

    for k in range(1, n + 1):
        pose = pyrosetta.pose_from_pdb(input_pdb)

        tf = build_task_factory_lock_receptor_and_cys(pose, peptide_chain_id=2)
        fd = FastDesign()
        fd.set_scorefxn(pyrosetta.get_score_function())
        fd.set_task_factory(tf)
        fd.apply(pose)

        new_seq = peptide_seq(pose, 2)
        diffs = diff_positions(orig_seq, new_seq)
        # 허용 design 포지션 밖 변이가 생기면 표시(데모에서는 이걸 필터로 활용)
        outside = [p for p in diffs if (p not in allowed and orig_seq[p - 1] != "C")]

        dG, dSASA = analyze_interface(pose)
        stab = stability_pk_proxy_scores(new_seq)

        out_name = f"candidate_{k:03d}.pdb"
        out_path = os.path.join("candidates", out_name)
        pose.dump_pdb(out_path)

        rows.append({
            "candidate": out_name,
            "pdb_path": out_path,
            "seq": new_seq,
            "dG_REU": dG,
            "dSASA": dSASA,
            "mut_positions": diffs,
            "mut_outside_allowed": outside,
            **stab
        })

    df = pd.DataFrame(rows)
    # 단순 멀티오브젝티브 랭킹(데모용)
    # Multi-objective rank score (데모용 가중치)
    # -dG: 결합 에너지가 낮을수록(음수) 좋으므로 부호 반전
    # cleavage_risk: 프로테아제 절단 위험 (K/R, 방향족 기반) — 낮을수록 좋음
    # pk_penalty: 약동학 불리 요소 (소수성 과다, 전하 불균형) — 낮을수록 좋음
    df["rank_score"] = (-df["dG_REU"]) - 0.5 * df["cleavage_risk"] - 1.0 * df["pk_penalty"]
    df = df.sort_values(["rank_score"], ascending=False).reset_index(drop=True)
    return df, orig_seq, design_positions

df_candidates, original_pep, design_positions = fastdesign_candidates(n=20, design_pos="2,6,7,8,10")
print("Original peptide:", original_pep)
print("Design positions:", design_positions)
df_candidates.head(10)

# 9-1. mut_outside_allowed 후보 자동 제거(필터링)

데모 스토리 포인트:
- FastDesign이 만든 후보 중 “허용한 설계 포지션 밖 변이”가 생긴 후보는
  **정책 위반**으로 자동 탈락시키고,
- 이후 FlexPepDock / PyMOL 스냅샷은 “정책 준수 후보”만 대상으로 진행합니다.

In [ ]:
import pandas as pd

assert "df_candidates" in globals(), "df_candidates가 없습니다. FastDesign 후보 생성 셀을 먼저 실행하세요."
df = df_candidates.copy()

assert "mut_outside_allowed" in df.columns, "mut_outside_allowed 컬럼이 없습니다."

def is_violation(x):
    if isinstance(x, bool):
        return bool(x)
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return False
    if isinstance(x, (str, list, tuple, set, dict)):
        return len(x) > 0
    return True

df["__violation__"] = df["mut_outside_allowed"].apply(is_violation)

df_pass = df[df["__violation__"] == False].drop(columns=["__violation__"]).reset_index(drop=True)
df_fail = df[df["__violation__"] == True].drop(columns=["__violation__"]).reset_index(drop=True)

print(f"[필터링 결과] 전체 후보: {len(df)}")
print(f"  - 통과(정책 준수): {len(df_pass)}")
print(f"  - 탈락(mut_outside_allowed): {len(df_fail)}")

display(df_pass.head(10))

df_candidates_filtered = df_pass

# 10. FlexPepDock로 상위 후보 refine + 재스코어링

FastDesign이 만든 후보는 “생성” 단계,
FlexPepDock은 “정밀 검증/리파인” 단계입니다.

여기서는 후보 상위 TopK(예: 10개)를 refine하고,
refine 후 dG/dSASA가 어떻게 변하는지 확인합니다.

※ 데모에서 더 깔끔하게 하려면, **필터링 통과 후보(df_candidates_filtered)** 에서 TopK를 뽑아 refine 하세요.

In [ ]:
from pyrosetta.rosetta.protocols.flexpep_docking import FlexPepDockingProtocol

def flexpepdock_refine(df_in, topk=10):
    os.makedirs("refined", exist_ok=True)
    rows = []
    top = df_in.head(topk)

    for i, row in top.iterrows():
        in_pdb = row["pdb_path"]
        pose = pyrosetta.pose_from_pdb(in_pdb)

        fpd = FlexPepDockingProtocol()
        fpd.apply(pose)

        dG, dSASA = analyze_interface(pose)
        seq = peptide_seq(pose, 2)
        stab = stability_pk_proxy_scores(seq)

        out_name = os.path.basename(row["candidate"]).replace("candidate_", "refined_")
        out_path = os.path.join("refined", out_name)
        pose.dump_pdb(out_path)

        rows.append({
            "input": row["candidate"],
            "input_pdb": in_pdb,
            "output": out_name,
            "pdb_path": out_path,
            "seq": seq,
            "dG_REU": dG,
            "dSASA": dSASA,
            **stab,
            "mut_outside_allowed": row.get("mut_outside_allowed", None),
        })

    df = pd.DataFrame(rows)
    df["rank_score"] = (-df["dG_REU"]) - 0.5 * df["cleavage_risk"] - 1.0 * df["pk_penalty"]
    df = df.sort_values(["rank_score"], ascending=False).reset_index(drop=True)
    return df

# 필터링 통과 후보로 refine 권장
df_refined = flexpepdock_refine(df_candidates_filtered, topk=min(10, len(df_candidates_filtered)))
df_refined

In [ ]:
# =========================================================
# 10-0. Refined 후보 시각화 (dG vs. Stability)
# =========================================================
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["font.size"] = 12

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# --- Plot 1: dG distribution ---
ax = axes[0]
ax.barh(range(len(df_refined)), df_refined["dG_REU"], color="#4A90D9", edgecolor="white")
ax.set_yticks(range(len(df_refined)))
ax.set_yticklabels(df_refined["output"] if "output" in df_refined.columns else df_refined.index, fontsize=8)
ax.set_xlabel("dG (REU)")
ax.set_title("Binding Energy (dG)")
ax.invert_yaxis()
ax.axvline(x=dG0, color="red", linestyle="--", label=f"baseline={dG0:.1f}")
ax.legend(fontsize=9)

# --- Plot 2: dSASA distribution ---
ax = axes[1]
ax.barh(range(len(df_refined)), df_refined["dSASA"], color="#F5A623", edgecolor="white")
ax.set_yticks(range(len(df_refined)))
ax.set_yticklabels([""] * len(df_refined))
ax.set_xlabel("dSASA (Å²)")
ax.set_title("Interface Buried Area (dSASA)")
ax.invert_yaxis()
ax.axvline(x=dSASA0, color="red", linestyle="--", label=f"baseline={dSASA0:.1f}")
ax.legend(fontsize=9)

# --- Plot 3: Multi-objective scatter ---
ax = axes[2]
sc = ax.scatter(
    df_refined["dG_REU"],
    df_refined["pk_penalty"],
    c=df_refined["rank_score"],
    cmap="RdYlGn", s=80, edgecolor="black", linewidth=0.5
)
ax.set_xlabel("dG (REU) ← better")
ax.set_ylabel("PK Penalty → lower is better")
ax.set_title("Multi-objective: dG vs. PK")
plt.colorbar(sc, ax=ax, label="rank_score")

plt.tight_layout()
plt.savefig("refined_analysis.png", dpi=150, bbox_inches="tight")
plt.show()
print("[OK] refined_analysis.png saved")

# 10-1. Top3 후보 구조 시각화

2가지 렌더링 모드를 자동 선택합니다:

1. **PyMOL** (설치된 경우): 고해상도 PNG 스냅샷 → `pymol_snapshots/`
2. **py3Dmol** (폴백): 노트북 인라인 3D 인터랙티브 뷰

- chain A(receptor): lightblue
- chain B(peptide): orange, stick 표시

In [ ]:
# =========================================================
# [구조 시각화] Top3 후보 — PyMOL (있으면) / py3Dmol (폴백)
# =========================================================

from pathlib import Path
from IPython.display import display as ipy_display

PDB_COL  = "pdb_path"
SCORE_COL = "rank_score"
TOPK = 3

assert "df_refined" in globals(), "df_refined가 없습니다. FlexPepDock refine 셀을 먼저 실행하세요."
df_viz = df_refined.copy()
df_top = df_viz.sort_values(SCORE_COL, ascending=False).head(TOPK).reset_index(drop=True)

print(f"[Top{TOPK} 후보] (정렬 기준: {SCORE_COL})")
display(df_top[[PDB_COL, SCORE_COL] + [c for c in ["seq","dG_REU","dSASA"] if c in df_top.columns]])

# --- PyMOL 사용 가능 여부 확인 ---
PYMOL_AVAILABLE = False
try:
    from pymol import cmd as pymol_cmd
    PYMOL_AVAILABLE = True
    print("[INFO] PyMOL detected — PNG 스냅샷 모드")
except ImportError:
    print("[INFO] PyMOL 미설치 — py3Dmol 인라인 3D 뷰 폴백")

# --- MODE 1: PyMOL ---
if PYMOL_AVAILABLE:
    OUTDIR = Path("pymol_snapshots")
    OUTDIR.mkdir(exist_ok=True)

    def render_snapshot_pymol(pdb_path: str, out_png: str,
                              width=1600, height=1200, dpi=300):
        pymol_cmd.reinitialize()
        pymol_cmd.load(pdb_path, "complex")
        pymol_cmd.hide("everything", "all")
        pymol_cmd.show("cartoon", "all")
        pymol_cmd.color("lightblue", "chain A")
        pymol_cmd.color("orange", "chain B")
        pymol_cmd.show("sticks", "chain B")
        pymol_cmd.bg_color("white")
        pymol_cmd.orient("all")
        pymol_cmd.zoom("all")
        pymol_cmd.png(out_png, width=width, height=height, dpi=dpi, ray=1)
        return out_png

    for i, row in df_top.iterrows():
        pdb_path = str(row[PDB_COL])
        out_png = str(OUTDIR / f"top{i+1}_snapshot.png")
        print(f"[Render] {pdb_path} → {out_png}")
        render_snapshot_pymol(pdb_path, out_png)

    pymol_cmd.quit()
    print("[OK] PyMOL 스냅샷 완료")

# --- MODE 2: py3Dmol (폴백) ---
else:
    try:
        import py3Dmol

        for i, row in df_top.iterrows():
            pdb_path = str(row[PDB_COL])
            with open(pdb_path, "r") as f:
                pdb_data = f.read()

            view = py3Dmol.view(width=700, height=500)
            view.addModel(pdb_data, "pdb")
            # Chain A: receptor (cartoon, lightblue)
            view.setStyle({"chain": "A"}, {"cartoon": {"color": "lightblue"}})
            # Chain B: peptide (cartoon + stick, orange)
            view.setStyle({"chain": "B"}, {"cartoon": {"color": "orange"}, "stick": {"color": "orange"}})
            view.zoomTo()
            view.setBackgroundColor("white")
            print(f"\n=== Top {i+1}: {os.path.basename(pdb_path)} | dG={row.get('dG_REU','N/A'):.2f} REU ===")
            ipy_display(view.show())

    except ImportError:
        print("[WARN] py3Dmol도 미설치. 시각화 생략.")
        print("  설치: pip install py3Dmol")
        print("  또는: conda install -c conda-forge py3dmol")

# 11. Relax 이후에도 체인 표준화 유지 확인

표준화된 체인이 Relax 이후에도 유지되는지 요약표로 다시 확인합니다.

In [ ]:
summarize_pdb("standardized_relaxed.pdb", show_seq="head")

# 12. 정리

이 노트북 데모에서 보여준 것:

1) 체인 A/B가 입력 순서에 따라 뒤바뀔 수 있는 위험을 **표준화 단계**로 제거  
2) 펩타이드만 Relax하여 리셉터 변형 치팅을 줄임  
3) FastDesign으로 후보 20개를 만들고, 테이블로 스코어/서열을 즉시 확인  
4) `mut_outside_allowed`로 **정책 위반 후보 자동 탈락**  
5) FlexPepDock으로 상위 후보를 refine하여 “검증 단계”를 구현  
6) Top3 후보를 PyMOL로 스냅샷 자동 생성하여 시각적으로 결과를 제시  
7) 혈중 안정성 목표에 대응하는 최소 stability/PK proxy 레이어 포함  

추후 확장:
- SSTR1/3/5 오프타깃에도 동일 후보를 넣어 Selectivity score 계산  
- stability/PK proxy를 실제 도구/모델/실험 기반으로 고도화